In [14]:
import os
from google.cloud import storage
import pandas as pd
import numpy as np
from flask import Flask
import bs4
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
import datetime
import re
from unidecode import unidecode
import requests

In [ ]:
LINK = 'https://cilar.com.br/alugar/apartamento-no-13-andar-com-1-quarto-cristo-rei-curitiba/id/55903662/code/03687.001'
driver = webdriver.Chrome()
driver.get(LINK)
soup = bs4.BeautifulSoup(driver.page_source, 'html.parser')

In [18]:
LINK = 'https://www.apolar.com.br/alugar/curitiba/cristo-rei/alugar-residencial-apartamento-curitiba-cristo-rei-106359'
driver = webdriver.Chrome()
driver.get(LINK)

In [19]:
soup = bs4.BeautifulSoup(driver.page_source, 'html.parser')

In [20]:
', '.join([i.text for i in soup.find_all('ul',{'class','property-details'})[0]]).replace('\n                        ','')

'Cidade: Curitiba, Área Terreno: 84.53m², Área Total: 84.53m², Dormitórios: 2, Área Útil: 66.39m², Suítes: 1, Garagem: 1, Condomínio: Sim, Face: Norte'

In [3]:
BUCKET_NAME = 'busca-apartamentos-bucket'
FILE = '2024-02-27 - apartamentos - apolar.csv'
TEMP_FILE = 'local.csv'

storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)

blop = bucket.blob(blob_name= FILE).download_as_string()
with open(TEMP_FILE, 'wb') as f:
    f.write(blop)

df = pd.read_csv('local.csv')

In [ ]:

df['titulo'] = df['titulo'].str.strip()
df['endereco'] = df['endereco'].str.strip()
df['descricao'] = df['descricao'].str.strip().str.replace('  ', '').str.replace('\n',' ')

def busca_substring(substring, string_list):
    result = np.nan
    for s in string_list:
        if substring in s:
            try:
                result = re.findall(r'\s(\d{1,3}(?:[.,]\d{3})*(?:[.,]\d{2})?)', s)[0]
            except:
                result = s
            break
            
    return result

def separa_valores_imovel(string):

    # Padrao regex para encontrar nome e valor monetário
    padrao = r'(\w+)\sR\$\s(\d{1,3}(?:\.\d{3})*(?:,\d{2})?)'

    # Encontrar todas as correspondências na string
    correspondencias = re.findall(padrao, string)

    # Imprimir os resultados
    list_values = []
    for correspondencia in correspondencias:
        nome, valor = correspondencia
        list_values.append(f'{nome}: {valor}')
    
    return list_values

#### FEATURES #######

# Localidade
df['bairro'] = df['endereco'].apply(lambda x: x if isinstance(x,float) else x.split(', ')[2].split(' - ')[0])
df['cidade'] = 'Curitiba'

# Atributos do imóvel
df['area'] = df['atributos'].apply(lambda x: x if isinstance(x,float) else busca_substring('m²', x.split(', ')))
df['banheiros'] = df['atributos'].apply(lambda x: x if isinstance(x,float) else busca_substring('banheiro', x.split(', ')))
df['vagas_garagem'] = df['atributos'].apply(lambda x: x if isinstance(x,float) else busca_substring('vaga', x.split(', ')))
df['quartos'] = df['atributos'].apply(lambda x: x if isinstance(x,float) else busca_substring('quarto', x.split(', ')))
df['suites'] = df['atributos'].apply(lambda x: x if isinstance(x,float) else busca_substring('suite', x.split(', ')))

df['area'] = df['area'].apply(lambda x: np.nan if isinstance(x, float) else x.split(' ')[0]).astype('float64')
df['banheiros'] = df['banheiros'].apply(lambda x: np.nan if isinstance(x, float) else x.split(' ')[0]).astype('float64')
df['vagas_garagem'] = df['vagas_garagem'].apply(lambda x: np.nan if isinstance(x, float) else x.split(' ')[0]).astype('float64')
df['quartos'] = df['quartos'].apply(lambda x: np.nan if isinstance(x, float) else x.split(' ')[0]).astype('float64')
df['suites'] = df['suites'].apply(lambda x: np.nan if isinstance(x, float) else x.split(' ')[0]).astype('float64')

# Valores
df['aluguel'] = df['valores'].apply(lambda x: x if isinstance(x,float) else busca_substring('Aluguel',separa_valores_imovel(x)))
df['condominio'] = df['valores'].apply(lambda x: x if isinstance(x,float) else busca_substring('Condomínio',separa_valores_imovel(x)))
df['seguro_incendio'] = df['valores'].apply(lambda x: x if isinstance(x,float) else busca_substring('Incêndio',separa_valores_imovel(x)))
df['iptu'] = df['valores'].apply(lambda x: x if isinstance(x,float) else busca_substring('IPTU',separa_valores_imovel(x)))

df['aluguel'] = df['aluguel'].apply(lambda x: x if isinstance(x,float) else x.replace(',00', '').replace('.','').replace(',','.')).astype('float64')
df['condominio'] = df['condominio'].apply(lambda x: x if isinstance(x,float) else x.replace(',00', '').replace('.','').replace(',','.')).astype('float64')
df['seguro_incendio'] = df['seguro_incendio'].apply(lambda x: x if isinstance(x,float) else x.replace(',00', '').replace('.','').replace(',','.')).astype('float64')
df['iptu'] = df['iptu'].apply(lambda x: x if isinstance(x,float) else x.replace(',00', '').replace('.','').replace(',','.')).astype('float64')

# Detalhes do imóvel/condomínio
df['mobiliado'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'mobiliado' in unidecode(x.lower()) else 'Não')
df['piscina'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'piscina' in unidecode(x.lower()) else 'Não')
df['academia'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'academia' in unidecode(x.lower()) else 'Não')
df['sacada'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'sacada' in unidecode(x.lower()) else 'Não')
df['churrasqueira'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'churrasqueira' in unidecode(x.lower()) else 'Não')
df['salao_de_festas'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'salao de festas' in unidecode(x.lower()) else 'Não')

####### TRATAMENTO FINAL DE DADOS ######

## Preenchendo valores nulos
df['cidade'] = df['cidade'].fillna('Curitiba')
# df[['aluguel','condominio','seguro_incendio','iptu']] = df[['aluguel','condominio','seguro_incendio','iptu']].fillna(0)
# df[['area','quartos','suites','banheiros','vagas_garagem']] = df[['area','quartos','suites','banheiros','vagas_garagem']].fillna(0)

## Transformando dtypes
df['data_coleta'] = pd.to_datetime(df['data_coleta'])
df['cidade'] = df['cidade'].apply(lambda x: x if isinstance(x,float) else unidecode(x.capitalize())).astype('category')
df['bairro'] = df['bairro'].apply(lambda x: x if isinstance(x,float) else unidecode(x.capitalize())).astype('category')
df[['area','quartos','suites','banheiros','vagas_garagem']] = df[['area','quartos','suites','banheiros','vagas_garagem']].astype('float64')
df[['aluguel','condominio','seguro_incendio','iptu']] = df[['aluguel','condominio','seguro_incendio','iptu']].astype('float64')

# df['valor_total'] = df['aluguel'] + df['condominio'] + df['seguro_incendio'] + df['iptu']


columns_selected = [
'site',
'titulo',
'link',
'data_coleta',
'endereco',
'atributos',
'descricao',
'bairro',
'cidade',
'aluguel',
'condominio',
'seguro_incendio',
'iptu',
'area',
'quartos',
'suites',
'banheiros',
'vagas_garagem',
'mobiliado',
'piscina',
'academia',
'sacada',
'churrasqueira',
'salao_de_festas'
]

BUCKET_NAME = 'busca-apartamentos-trusted'
FILE_NAME = f'{datetime.datetime.today().strftime("%Y-%m-%d")} - apartamentos - apolar.csv'
TEMP_FILE = 'local.csv'
RAW_PATH = '/tmp/{}'.format(TEMP_FILE)

storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)

df[columns_selected].to_csv(RAW_PATH, index=False)
blob = bucket.blob(FILE_NAME)
blob.upload_from_filename(RAW_PATH)